# Clustering Crypto

In [46]:
# Initial imports
import pandas as pd
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Deliverable 1: Preprocessing the Data for PCA

In [47]:
# Load the crypto_data.csv dataset.
file = 'crypto_data.csv'
file
df = pd.read_csv(file)
df.set_index("Unnamed: 0", inplace=True)
df.index.name = None

df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [48]:
# Keep all the cryptocurrencies that are being traded.
clean_crypto_df = df.loc[df['IsTrading'] == True]
print(clean_crypto_df.shape)
clean_crypto_df.head(10)

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [49]:
# Keep all the cryptocurrencies that have a working algorithm.
clean_crypto_df[clean_crypto_df.isnull()["Algorithm"] == True] 
print(clean_crypto_df.shape)
clean_crypto_df.head(10)

(1144, 6)


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [50]:
# Remove the "IsTrading" column. 
clean_crypto_df = df.drop(columns='IsTrading')
print(clean_crypto_df.shape)
clean_crypto_df.head(10)

(1252, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [51]:
# Remove rows that have at least 1 null value.
clean_crypto_df = clean_crypto_df.dropna()
clean_crypto_df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [52]:
# Keep the rows where coins are mined.
clean_crypto_df = clean_crypto_df[clean_crypto_df["TotalCoinsMined"] > 0]
print(clean_crypto_df.shape)
clean_crypto_df.head(10)

(577, 5)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [53]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names_df = clean_crypto_df.drop(labels=["Algorithm", "ProofType", "TotalCoinsMined", "TotalCoinSupply"],
                                 axis=1)
crypto_names_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [54]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
clean_crypto_df.drop(labels="CoinName", axis=1, inplace=True)
print(clean_crypto_df.shape)
clean_crypto_df.head(10)


(577, 4)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [55]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(clean_crypto_df, columns=["Algorithm", "ProofType"])
print(X.shape)
X.head(10)

(577, 101)


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEC,7.383056e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [56]:
# Standardize the data with StandardScaler().
X_scaled = StandardScaler().fit_transform(X)
X_scaled

array([[-0.11450078, -0.15072489, -0.04166667, ..., -0.04166667,
        -0.04166667, -0.04166667],
       [-0.09041991, -0.14255828, -0.04166667, ..., -0.04166667,
        -0.04166667, -0.04166667],
       [ 0.55369883,  4.67185916, -0.04166667, ..., -0.04166667,
        -0.04166667, -0.04166667],
       ...,
       [-0.11433426, -0.15040252, -0.04166667, ..., -0.04166667,
        -0.04166667, -0.04166667],
       [-0.11449785, -0.15070954, -0.04166667, ..., -0.04166667,
        -0.04166667, -0.04166667],
       [-0.11401032, -0.14918981, -0.04166667, ..., -0.04166667,
        -0.04166667, -0.04166667]])

### Deliverable 2: Reducing Data Dimensions Using PCA

In [57]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components = 3)
X_pca = pca.fit_transform(X_scaled)
X_pca

array([[-0.32239404,  1.10701341, -0.39230092],
       [-0.30504554,  1.10711641, -0.39259495],
       [ 2.43399124,  1.64368697, -0.45696811],
       ...,
       [-0.14313167, -2.16366697,  0.44960825],
       [-0.28061881,  0.80332127, -0.19925006],
       [-0.32138379,  1.10696861, -0.39230859]])

In [58]:
# Create a DataFrame with the three principal components.
df_pca = pd.DataFrame(data=X_pca, columns=["principal component 1", "principal component 2","principal component 3"], index=X.index)
df_pca.head(10)

,principal component 1,principal component 2,principal component 3
42,-0.322394,1.107013,-0.392301
404,-0.305046,1.107116,-0.392595
1337,2.433991,1.643687,-0.456968
BTC,-0.137322,-1.308820,0.137735
ETH,-0.142788,-2.039796,0.292864
LTC,-0.160631,-1.051582,0.015252
DASH,-0.386449,1.213430,-0.422263
XMR,-0.156751,-2.244402,0.265103
ETC,-0.141169,-2.039904,0.292859
ZEC,-0.143131,-2.163667,0.449608


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [59]:
# Create an elbow curve to find the best value for K.
inertia = []
K = list(range(1, 11))

# Calculate the inertia for the range of K values
for k in K:
    km = KMeans(n_clusters=k)
    km.fit(df_pca)
    inertia.append(km.inertia_)

# Create Elbow curve
elbow_data = {"K": K, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="K", y="inertia", xticks=K, title="Elbow Curve")  # Elbow at K = 4

:Curve   [K]   (inertia)

Running K-Means with `k=4`

In [60]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4)

# Fit the model
model.fit(df_pca)

# Predict clusters
predictions = model.predict(df_pca)
predictions

array([2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2,
       0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0,
       0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 2,
       2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 2, 0, 2, 0, 2, 0,
       2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2,
       0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2,
       2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 2, 0, 2, 2, 2,
       2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 0, 2, 0,
       2, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 0, 0, 2, 0, 2, 2,
       0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0,

In [61]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = clean_crypto_df.join(df_pca)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = clustered_df.join(crypto_names_df)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df["Class"] = model.labels_

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(577, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,principal component 1,principal component 2,principal component 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.322394,1.107013,-0.392301,42 Coin,2
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.305046,1.107116,-0.392595,404Coin,2
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.433991,1.643687,-0.456968,EliteCoin,2
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.137322,-1.308820,0.137735,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.142788,-2.039796,0.292864,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.160631,-1.051582,0.015252,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.386449,1.213430,-0.422263,Dash,2
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.156751,-2.244402,0.265103,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.141169,-2.039904,0.292859,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.143131,-2.163667,0.449608,ZCash,0


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [70]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(clustered_df, x="principal component 1", y="principal component 2", z="principal component 3",
                    color="Class", symbol="Class", width=800, hover_name="CoinName",
                    hover_data=["Algorithm"])
fig.show()

In [63]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType",
                                   "TotalCoinSupply", "TotalCoinsMined", "Class"])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [71]:
# Print the total number of tradable cryptocurrencies.
clustered_df.shape[0]


577

In [65]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.

X_tradable = clustered_df[["TotalCoinSupply", "TotalCoinsMined"]]
X_tradable.head(10)


,TotalCoinSupply,TotalCoinsMined
42,42,4.199995e+01
404,532000000,1.055185e+09
1337,314159265359,2.927942e+10
BTC,21000000,1.792718e+07
ETH,0,1.076842e+08
LTC,84000000,6.303924e+07
DASH,22000000,9.031294e+06
XMR,0,1.720114e+07
ETC,210000000,1.133597e+08
ZEC,21000000,7.383056e+06


In [66]:
X_tradable_scaled = MinMaxScaler().fit_transform(X_tradable)
X_tradable_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07],
       [1.00000000e-04, 2.17085015e-05]])

In [67]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(data=X_tradable_scaled, columns=["TotalCoinSupply", "TotalCoinsMined"],
                       index=X_tradable.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df["CoinName"] = clustered_df["CoinName"]

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df["Class"] = clustered_df["Class"]

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.000000,42 Coin,2
404,5.320000e-04,0.001066,404Coin,2
1337,3.141593e-01,0.029576,EliteCoin,2
BTC,2.100000e-05,0.000018,Bitcoin,0
ETH,0.000000e+00,0.000109,Ethereum,0
LTC,8.400000e-05,0.000064,Litecoin,0
DASH,2.200000e-05,0.000009,Dash,2
XMR,0.000000e+00,0.000017,Monero,0
ETC,2.100000e-04,0.000115,Ethereum Classic,0
ZEC,2.100000e-05,0.000007,ZCash,0


In [68]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", hover_cols=["CoinName"], by="Class")


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)